<a href="https://colab.research.google.com/github/Sarthak9504/SY-Sem1-CourseProjects/blob/EDAI-3/Marathi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [86]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, f1_score
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras import layers
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import tensorflow as tf
import re
from google.colab import drive
from sklearn.model_selection import train_test_split

In [87]:
df=pd.read_excel("/content/tweets-train-cleaned.xlsx", index_col='Unnamed: 0')
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [88]:

df.loc[~df['tweet'].isnull()]

,tweet,label
0,ज्येष्ठ पत्रकार अनंत दीक्षित यां निधन वृत्त दु...,-1
1,सर्वोच् न्ल निर्देश डावलून पुण पोलिस प्र आनंद ...,-1
2,उद्ध ठाकर भाजपासोबत युत शिसैनिक अमान िश्ासघ फस...,-1
3,आपल समाज खूप मोठ त्य घटक अंतर्भूत त या दिव्यां...,1
4,बलात्कार बदल बलात्कार घेतल पाहिज म्हणणार सावरक...,-1
...,...,...
12109,त्याचबरोबर काज व्यावसायिकां मागील कालावध व्हॅट...,1
12110,सरकार कर्जमाफ योज लाग करुन सुमार वर्ष काळ उलटू...,-1
12111,लोकश मतदार खऱ्य अर्थ सत्ताधार असत धमकािण अथ प्...,-1
12112,छत्रपत शिवाज महाराज ंच् नाव शिवसेने स्थाप ंन श...,-1


In [89]:
df.shape

(12114, 2)

In [90]:
df.loc[~df['tweet'].isnull()]

,tweet,label
0,ज्येष्ठ पत्रकार अनंत दीक्षित यां निधन वृत्त दु...,-1
1,सर्वोच् न्ल निर्देश डावलून पुण पोलिस प्र आनंद ...,-1
2,उद्ध ठाकर भाजपासोबत युत शिसैनिक अमान िश्ासघ फस...,-1
3,आपल समाज खूप मोठ त्य घटक अंतर्भूत त या दिव्यां...,1
4,बलात्कार बदल बलात्कार घेतल पाहिज म्हणणार सावरक...,-1
...,...,...
12109,त्याचबरोबर काज व्यावसायिकां मागील कालावध व्हॅट...,1
12110,सरकार कर्जमाफ योज लाग करुन सुमार वर्ष काळ उलटू...,-1
12111,लोकश मतदार खऱ्य अर्थ सत्ताधार असत धमकािण अथ प्...,-1
12112,छत्रपत शिवाज महाराज ंच् नाव शिवसेने स्थाप ंन श...,-1


In [91]:
df_v=pd.read_excel("/content/tweets-valid-cleaned.xlsx", index_col='Unnamed: 0')

In [92]:
df_v.loc[~df_v['tweet'].isnull()]

,tweet,label
0,कोर काळ पंतप्रधान मा नरेंद्र मोदीज य समाज घटक ...,1
1,पुण राष्ट्रसे दल अभ्सिक िभाग ंच् संयुक्त िद्यम...,1
2,पुण क्रांत ज्योत कार्यक्रम जेष्ठ समाजवाद ने मा...,0
3,आद्यक्रांतिकारक उमाजीराज नाईक य जयंतीनिमित्त व...,1
4,महाराष्ट्र मुख्यमंत्र शपथ घेतल्याबद्दल शिवस पक...,1
...,...,...
1495,पोलिसभरत प्रक्रिय जु निकषांनुसार व्व याबाबत मा...,1
1496,निध उद्ध्वस्त कुटुंबांमधील मुल शिक्षण वापरल जा...,1
1497,िकासा फक्त घोषणाबाज जाहिरातबाज राज्य जनते दिशा...,0
1498,विधानसभेत राष्ट्रीय जनगण ओबीसीं इतर मागासवर्गी...,0


In [93]:
df_v.shape

(1500, 2)

In [94]:
df_t = pd.read_excel("/content/tweets-test-cleaned.xlsx", index_col='Unnamed: 0')

In [95]:
df_t.shape

(2250, 2)

In [96]:
max(df_t['tweet'].str.len())

240

In [97]:
max(df_v['tweet'].str.len())

245

In [98]:
vocab_df=pd.concat([df_v['tweet'], df.loc[~df['tweet'].isnull(), 'tweet']], axis = 0)

In [99]:
vocab_df.isnull().sum()

0

In [100]:
set2=set()
for row in vocab_df.iteritems():
  for words in row[1].split():
    set2.add(words)
len(set2)

<ipython-input-100-c0937d06b7d9>:2: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for row in vocab_df.iteritems():


38045

In [101]:

vocab_size=40000


In [102]:
from collections import Counter
def counter_word(text_col):
  count=Counter()
  for text in text_col.values:
    for word in text.split():
      count[word]+=1
  return count

In [103]:

counter = counter_word(vocab_df)

len(counter)


38045

In [104]:
df = df[df['label'] != 2]
df_v = df_v[df_v['label'] != 2]
df_t = df_t[df_t['label'] != 2]

In [105]:

from tensorflow.keras.preprocessing.text import Tokenizer
tokinizer=Tokenizer(num_words=40000)
tokinizer.fit_on_texts(df.loc[~df['tweet'].isnull(), 'tweet'])

In [106]:

word_index = tokinizer.word_index

In [107]:
word_index

{'य': 1,
 'सरकार': 2,
 'महाराष्ट्र': 3,
 'यां': 4,
 'आ': 5,
 'राज्य': 6,
 '…': 7,
 'करण्य': 8,
 'भाजप': 9,
 'कर': 10,
 'काँग्रेस': 11,
 'करत': 12,
 'शुभेच्छ': 13,
 'त्य': 14,
 'त': 15,
 'राज': 16,
 'घेतल': 17,
 'उपस्थित': 18,
 'दि': 19,
 'मा': 20,
 'देश': 21,
 'पक्ष': 22,
 'अभिवादन': 23,
 'सुर': 24,
 'मुंबई': 25,
 'विनम्र': 26,
 'श्र': 27,
 'मुख्यमंत्र': 28,
 'भेट': 29,
 'मराठ': 30,
 'करणार': 31,
 'या': 32,
 'शेतकर': 33,
 'शेतकऱ्य': 34,
 'वर्ष': 35,
 'जन': 36,
 'ने': 37,
 'हार्दिक': 38,
 'श्रद्धांजल': 39,
 'माज': 40,
 'निर्णय': 41,
 'सर्व': 42,
 'लोक': 43,
 'मदत': 44,
 'मंत्र': 45,
 'ना': 46,
 'अस': 47,
 'असत': 48,
 'माझ': 49,
 'च्य': 50,
 'मागण': 51,
 'केंद्र': 52,
 'पाहिज': 53,
 'मोद': 54,
 'महाराज': 55,
 'प्रश्न': 56,
 'साहेब': 57,
 'राष्ट्रवाद': 58,
 'अश': 59,
 'चर्': 60,
 'काळ': 61,
 'भावपूर्ण': 62,
 'भाग': 63,
 'करण': 64,
 'समाज': 65,
 'अध्यक्ष': 66,
 'घेत': 67,
 'कोर': 68,
 'मात्र': 69,
 'बैठक': 70,
 'मोठ': 71,
 'जिल्ह्य': 72,
 'पुण': 73,
 'माझ्य': 74,
 'कराव': 75,
 'ंन': 76,
 '

In [108]:
train_sequences = tokinizer.texts_to_sequences(df.loc[~df['tweet'].isnull(), 'tweet'])

In [109]:
test_sequences = tokinizer.texts_to_sequences(df_t['tweet'].values)


In [110]:
val_sequences = tokinizer.texts_to_sequences(df_v['tweet'].values)


In [111]:

print(df['tweet'][0])
print(train_sequences[0])

ज्येष्ठ पत्रकार अनंत दीक्षित यां निधन वृत्त दुखद दशकं परखड लेखण पत्रकारितेत अमूल्य योगदान दि दीक्षित यां मार्गदर्शनाखाल पत्रकारां पिढ घडल अनंत दीक्षित य भावपूर्ण श्रद्धांजल त्य परिवार दुख सहभाग आहोत
[83, 132, 1800, 4171, 4, 77, 795, 1676, 8272, 2692, 2387, 10711, 2693, 268, 19, 4171, 4, 1864, 5150, 994, 1732, 1800, 4171, 1, 62, 39, 14, 1733, 1230, 97, 236]


In [112]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [113]:
train_padded=pad_sequences(train_sequences, maxlen=250, padding='post', truncating='post')
val_padded=pad_sequences(val_sequences, maxlen=250, padding='post', truncating='post')
test_padded=pad_sequences(test_sequences, maxlen=250, padding='post', truncating='post')

In [114]:
print(train_padded.shape, val_padded.shape, test_padded.shape)

(12112, 250) (1500, 250) (2250, 250)


In [115]:
from keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.metrics import classification_report
from keras.optimizers import Adam
model = Sequential()

In [116]:
model.add(layers.Embedding(40000, 32, input_length=250))
model.add(layers.Dense(256, activation='gelu'))
model.add(layers.Flatten())
model.add(layers.Dense(3, activation='softmax'))

In [117]:
optim = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy'])


In [118]:
train_label_categorical = to_categorical(df.loc[~df['tweet'].isnull(), 'label'], num_classes=3)


val_label_categorical = to_categorical(df_v['label'].values, num_classes=3)


test_label_categorical = to_categorical(df_t['label'].values, num_classes=3)

In [119]:

val_label_categorical

array([[0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.]], dtype=float32)

In [120]:
train_label_categorical

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [121]:
tf.config.run_functions_eagerly(True)
model.fit(train_padded, to_categorical(df.loc[~df['tweet'].isnull(), 'label'].values, num_classes=3), epochs=5, validation_data=(val_padded, to_categorical(df_v['label'].values, num_classes=3)), verbose=2)

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


379/379 - 16s - loss: 1.0698 - accuracy: 0.4132 - val_loss: 1.0203 - val_accuracy: 0.4680 - 16s/epoch - 43ms/step
Epoch 2/5


KeyboardInterrupt: ignored

In [122]:
!pip install transformers


In [123]:
!pip install sentence-transformers


In [124]:
from transformers import AutoTokenizer, BertTokenizer, TFBertModel, AutoModel, TFAutoModel, TFAutoModelForSequenceClassification

In [125]:
from sentence_transformers import SentenceTransformer

In [126]:
tokenizer = AutoTokenizer.from_pretrained("l3cube-pune/indic-sentence-bert-nli")

In [127]:
df = pd.read_excel("/content/tweets-train-cleaned.xlsx", index_col='Unnamed: 0')
df_v = pd.read_excel("/content/tweets-valid-cleaned.xlsx", index_col='Unnamed: 0')
df_t = pd.read_excel("/content/tweets-test-cleaned.xlsx", index_col='Unnamed: 0')


In [128]:
train_inputs = tokenizer(df.loc[~df['tweet'].isnull(), 'tweet'].tolist(), padding=True, truncation=True, max_length=250, return_tensors="tf")
val_inputs = tokenizer(df_v['tweet'].tolist(), padding=True, truncation=True, max_length=250, return_tensors="tf")
test_inputs = tokenizer(df_t['tweet'].tolist(), padding=True, truncation=True, max_length=250, return_tensors="tf")

In [129]:
train_labels = df.loc[~df['tweet'].isnull(), 'label'].values
val_labels = df_v['label'].values
test_labels = df_t['label'].values

In [130]:
num_classes = 3
train_label_categorical = to_categorical(train_labels, num_classes=num_classes)
val_label_categorical = to_categorical(val_labels, num_classes=num_classes)
test_label_categorical = to_categorical(test_labels, num_classes=num_classes)

In [131]:
model = SentenceTransformer("l3cube-pune/indic-sentence-bert-nli")

In [132]:
train_embeddings = model.encode(df.loc[~df['tweet'].isnull(), 'tweet'].tolist(), show_progress_bar=True)
val_embeddings = model.encode(df_v['tweet'].tolist(), show_progress_bar=True)
test_embeddings = model.encode(df_t['tweet'].tolist(), show_progress_bar=True)

Batches:   0%|          | 0/379 [00:00<?, ?it/s]

Batches:   0%|          | 0/47 [00:00<?, ?it/s]

Batches:   0%|          | 0/71 [00:00<?, ?it/s]

In [133]:
classification_model = tf.keras.Sequential([
    Dense(256, activation='relu', input_shape=(train_embeddings.shape[1],)),
    Dropout(0.2),
    Dense(num_classes, activation='softmax')
])

In [134]:
optim = tf.keras.optimizers.Adam(learning_rate=0.0001)
classification_model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy'])


In [144]:
classification_model.fit(
    train_embeddings,
    train_label_categorical,
    epochs=9,
    batch_size=32,
    validation_data=(val_embeddings, val_label_categorical),
    verbose=2
)

Epoch 1/9


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


379/379 - 11s - loss: 0.5930 - accuracy: 0.7531 - val_loss: 0.5647 - val_accuracy: 0.7760 - 11s/epoch - 29ms/step
Epoch 2/9
379/379 - 11s - loss: 0.5882 - accuracy: 0.7545 - val_loss: 0.5624 - val_accuracy: 0.7720 - 11s/epoch - 29ms/step
Epoch 3/9
379/379 - 12s - loss: 0.5831 - accuracy: 0.7579 - val_loss: 0.5588 - val_accuracy: 0.7760 - 12s/epoch - 33ms/step
Epoch 4/9
379/379 - 12s - loss: 0.5806 - accuracy: 0.7577 - val_loss: 0.5560 - val_accuracy: 0.7753 - 12s/epoch - 32ms/step
Epoch 5/9
379/379 - 12s - loss: 0.5792 - accuracy: 0.7573 - val_loss: 0.5552 - val_accuracy: 0.7727 - 12s/epoch - 31ms/step
Epoch 6/9
379/379 - 10s - loss: 0.5757 - accuracy: 0.7602 - val_loss: 0.5529 - val_accuracy: 0.7767 - 10s/epoch - 27ms/step
Epoch 7/9
379/379 - 12s - loss: 0.5725 - accuracy: 0.7642 - val_loss: 0.5502 - val_accuracy: 0.7753 - 12s/epoch - 33ms/step
Epoch 8/9
379/379 - 12s - loss: 0.5692 - accuracy: 0.7627 - val_loss: 0.5475 - val_accuracy: 0.7787 - 12s/epoch - 31ms/step
Epoch 9/9
379/379 

In [145]:
test_loss, test_acc = classification_model.evaluate(test_embeddings, test_label_categorical)
print("Test accuracy:", test_acc)

71/71 [==============================] - 1s 19ms/step - loss: 0.5673 - accuracy: 0.7729
Test accuracy: 0.7728888988494873


In [137]:
predictions = classification_model.predict(test_embeddings)
predicted_labels = np.argmax(predictions, axis=1)

71/71 [==============================] - 0s 3ms/step


In [146]:
print(classification_report(test_labels, predicted_labels))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       750
           0       0.72      0.73      0.73       750
           1       0.80      0.72      0.76       750
           2       0.00      0.00      0.00         0

    accuracy                           0.48      2250
   macro avg       0.38      0.36      0.37      2250
weighted avg       0.51      0.48      0.49      2250



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

In [148]:
predicted_labels = np.argmax(predictions, axis=1)

In [147]:
conf_matrix = confusion_matrix(test_labels, predicted_labels, labels=[0, 1, 2])

In [149]:
print("Confusion Matrix:")
print(conf_matrix)

Confusion Matrix:
[[551  90 109]
 [136 538  76]
 [  0   0   0]]


In [150]:
f1_macro = f1_score(test_labels, predicted_labels, average='macro')
f1_micro = f1_score(test_labels, predicted_labels, average='micro')

In [151]:
print("Macro F1 Score:", f1_macro)
print("Micro F1 Score:", f1_micro)

Macro F1 Score: 0.37067262945222584
Micro F1 Score: 0.484
